# Домашнее задание «Библиотека Pandas»

<a id='contents'></a>
## Оглавление:
### [Задание 1](#task1)
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class
### [Задание 2](#task2)
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

    geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
    }
Результат классификации запишите в отдельный столбец region.
### [Задание 3](#task3)
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы grouplens.org.../movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

P/S/ все файлы находятся в архиве «дополнительные файлы к лекции и дз»

<a id='task1'></a>
## Задание 1

In [156]:
import pandas as pd

path = r'D:\Google Disk\Учеба\Data Scientist с нуля до middle\Python\Материалы курса\NUMPY-21\Python_11_доп_файлы\ratings.csv'
# загружаем датафрейм с данными по рейтингам фильмов от пользователей
df_rate = pd.read_csv(path, encoding = 'utf-8')
df_rate.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [157]:
path = r'D:\Google Disk\Учеба\Data Scientist с нуля до middle\Python\Материалы курса\NUMPY-21\Python_11_доп_файлы\movies.csv'
# загружаем датафрем с данными о фильмах
df_film = pd.read_csv(path, encoding = 'utf-8')
df_film.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [159]:
# объединяем рейтинги с фильмами
df_cons = pd.merge(df_film, df_rate, left_on='movieId', right_on='movieId')

In [160]:
# удаляем лишние поля применяя операцию в датафрейму через inplace
df_cons.drop(columns=['genres', 'userId', 'timestamp'], inplace = True)

In [161]:
# считаем средний среди пользователей рейтинг у каждого фильма для последующей классификации
df_cons = df_cons.groupby(['movieId', 'title']).mean().reset_index() # т.к. при группировке столбец, по которому
                                                                     # группировали стал индексом, из-за чего будет размещен
                                                                     # на уровень ниже остальных - применяем reset_index() для
                                                                     # переиндексации датафрейма
df_cons.head()

,movieId,title,rating
0,1,Toy Story (1995),3.872470
1,2,Jumanji (1995),3.401869
2,3,Grumpier Old Men (1995),3.161017
3,4,Waiting to Exhale (1995),2.384615
4,5,Father of the Bride Part II (1995),3.267857


In [162]:
df_cons['rating'] = df_cons['rating'].round(1)
df_cons.head()

,movieId,title,rating
0,1,Toy Story (1995),3.9
1,2,Jumanji (1995),3.4
2,3,Grumpier Old Men (1995),3.2
3,4,Waiting to Exhale (1995),2.4
4,5,Father of the Bride Part II (1995),3.3


In [163]:
def film_class (in_rate):
    '''
    функция оценивает средний рейтинг фильма и
    передает его словесное описание
    '''
    if in_rate <= 2:
        return 'low ratign'
    elif in_rate <= 4:
        return 'medium rating'
    else:
        return 'high rating'

In [191]:
df_cons['class'] = df_cons['rating'].apply(film_class)
df_cons = df_cons.sort_values(by = 'rating', ascending = False)
df_cons.head()

,movieId,title,rating,class
9065,163949,The Beatles: Eight Days a Week - The Touring Y...,5.0,high rating
4787,6769,Mambo Italiano (2003),5.0,high rating
4464,6107,Night of the Shooting Stars (Notte di San Lore...,5.0,high rating
4487,6163,He Loves Me... He Loves Me Not (À la folie... ...,5.0,high rating
4580,6332,"Lizzie McGuire Movie, The (2003)",5.0,high rating


In [197]:
df_cons.groupby('class').agg({'movieId' : ['count'], 'rating' : ['min', 'max']}).reset_index()

class movieId rating     
                   count    min  max
0    high rating    1212    4.1  5.0
1     low ratign     979    0.5  2.0
2  medium rating    6875    2.1  4.0

[к оглавлению](#contents)

<a id='task2'></a>
## Задание 2

In [138]:
import pandas as pd

# загружаем датафрейм с данными по регионам
path = r'D:\Google Disk\Учеба\Data Scientist с нуля до middle\Python\Материалы курса\NUMPY-21\Python_11_доп_файлы\keywords.csv'

df_rate = pd.read_csv(path, encoding = 'utf-8')
df_rate.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [139]:
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [140]:
geo_data

{'Центр': ['москва', 'тула', 'ярославль'],
 'Северо-Запад': ['петербург', 'псков', 'мурманск'],
 'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}

In [141]:
def check_regs (request):
    for reg, cities in geo_data.items():
        for city in cities:
            pattern = r'.*' + city + '.*'
            if re.search(pattern, request.lower()): # применяем .lower(), чтобы все буквы сделать в нижнем регистре
                return reg
    return 'undefined'

In [142]:
import re

df_rate['region'] = df_rate['keyword'].apply(check_regs)
df_rate.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [144]:
df_rate.groupby(['region']).count().reset_index()

,region,keyword,shows
0,undefined,99253,99253
1,Дальний Восток,94,94
2,Северо-Запад,266,266
3,Центр,387,387


[к оглавлению](#contents)

<a id='task3'></a>
## Задание 3

In [253]:
import pandas as pd
import numpy as np

path = r'D:\Google Disk\Учеба\Data Scientist с нуля до middle\Python\Материалы курса\NUMPY-21\Python_11_доп_файлы\ratings.csv'
# загружаем датафрейм с данными по рейтингам фильмов от пользователей
df_rate = pd.read_csv(path, encoding = 'utf-8')

In [254]:
path = r'D:\Google Disk\Учеба\Data Scientist с нуля до middle\Python\Материалы курса\NUMPY-21\Python_11_доп_файлы\movies.csv'
# загружаем датафрем с данными о фильмах
df_film = pd.read_csv(path, encoding = 'utf-8')

In [272]:
df_cons = pd.merge(df_film, df_rate, left_on = 'movieId', right_on = 'movieId')
df_cons.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,3.0,851866703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9,4.0,938629179
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13,5.0,1331380058
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.0,997938310
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19,3.0,855190091


In [273]:
years = list(np.arange(1950, 2010 + 1)) # генерируем список годов

In [274]:
def production_year(in_film):
    for year in years:
        pattern = r'.*' + str(year) + '.*'
        if re.search(pattern, in_film):
            return year
    return '1900'    

In [275]:
df_cons['year'] = df_cons['title'].apply(production_year)

In [276]:
df_cons.head()

,movieId,title,genres,userId,rating,timestamp,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,3.0,851866703,1995
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9,4.0,938629179,1995
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13,5.0,1331380058,1995
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.0,997938310,1995
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19,3.0,855190091,1995


In [277]:
df_cons = df_cons.groupby('year').mean()['rating'].reset_index()
df_cons.head()

,year,rating
0,1950,3.915254
1,1951,3.983539
2,1952,4.000000
3,1953,3.754587
4,1954,3.994220


In [280]:
df_cons.sort_values(by = 'rating', ascending = False).reset_index(drop = True)

,year,rating
0,1957,4.014241
1,1972,4.011136
2,1952,4.000000
3,1954,3.994220
4,1951,3.983539
...,...,...
57,2005,3.448434
58,2003,3.444777
59,1996,3.426600
60,1997,3.415764


[к оглавлению](#contents)